# Algorithmic Trading

## Introduction

In the world of financial markets, investors employ various strategies to optimize their portfolio returns. Two popular approaches are momentum and value investing. In this context, we will explore the steps involved in creating and implementing these strategies using S&P 500 stocks. **

**Strategy Development Process:**

The process of running an investing strategy is divided into three steps -

1] Collect data 

2] Develop a hypothesis for a strategy 

3] Backtest the strategy 

4] Implement the strategy in production

In [57]:
# Here are several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 

# Fetch file paths
import os
for dirname, _, filenames in os.walk('/path/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/algorithmic-trading-dataset/value_strategy_1.csv.csv
/kaggle/input/algorithmic-trading-dataset/recommended_trades_1.csv.csv
/kaggle/input/algorithmic-trading-dataset/momentum_strategy_1.csv.csv
/kaggle/input/sp-500-stocks/sp_500_stocks.csv


In [58]:
#Getting data from csv files.
value_strategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/value_strategy_1.csv.csv")
recommended_trades = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/recommended_trades_1.csv.csv")
momentum_srategy = pd.read_csv("/kaggle/input/algorithmic-trading-dataset/momentum_strategy_1.csv.csv")
trades = pd.read_csv("/kaggle/input/sp-500-stocks/sp_500_stocks.csv")

In [59]:
momentum_srategy.head()
#len(momentum_srategy)

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,FANG,$102.81,167.05%,98.42%,110.39%,99.80%,23.90%,95.84%,15.58%,97.23%,97.82%,786
1,NVDA,$836.32,117.59%,92.67%,57.08%,96.24%,47.60%,100.00%,25.29%,99.60%,97.13%,96
2,DXC,$42.36,156.16%,97.43%,58.69%,96.83%,31.47%,99.01%,6.28%,83.56%,94.21%,1909
3,OXY,$32.72,92.75%,85.54%,91.00%,99.21%,20.26%,92.28%,15.90%,97.43%,93.61%,2472
4,IT,$258.87,107.52%,91.09%,57.23%,96.44%,32.85%,99.60%,6.90%,84.75%,92.97%,312


In [60]:
value_strategy.head()

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Value Score,Number of Shares to Buy
0,UNM,$29.04,7.77,13.3%,0.56,8.7%,0.46,2.2%,3.60,4.8%,0.43,1.0%,6.0%,83358
1,MCK,$195.80,-7.06,9.9%,-1507.20,0.4%,0.13,1.0%,7.57,9.3%,2.61,10.5%,6.2%,12363
2,AIV,$7.04,-76.61,2.4%,2.03,29.9%,-31.76,0.2%,-44.29,0.8%,-53.16,0.6%,6.8%,343852
3,AIG,$48.50,-11.09,8.7%,0.67,9.1%,0.96,8.5%,6.09,6.5%,0.94,2.6%,7.1%,49911
4,L,$56.64,-409.90,0.4%,0.88,9.9%,1.04,9.9%,8.29,13.5%,1.02,2.8%,7.3%,42738


In [61]:
recommended_trades.head()
#len(recommended_trades)

,Ticker,Stock Price,Market Capitalization,Number of Share to Buy
0,A,$155.08,4.705865e+10,18.0
1,AAL,$22.02,1.369795e+10,130.0
2,AAP,$213.33,1.372993e+10,13.0
3,AAPL,$141.25,2.343545e+12,20.0
4,ABBV,$115.40,2.054323e+11,24.0


In [62]:
trades.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Data Collection and IEX Cloud API Integration

In this project, we are using the IEX Cloud API to retrieve real-time stock prices and other relevant financial information. This integration allows us to access up-to-date data for informed decision-making and analysis.

In [63]:
import requests
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
IEX_CLOUD_API_TOKEN = "token"
secret_value_0 = user_secrets.get_secret("IEX_CLOUD_API_TOKEN")
symbol = 'AAPL'
#Getting the data of S&P stocks and storing it.
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 89715851,
 'calculationPrice': 'tops',
 'change': 3.697,
 'changePercent': 0.02276,
 'close': 0,
 'closeSource': 'fliocfai',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': 'rime cleaIt EpirX e',
 'highTime': 1648736386083,
 'iexAskPrice': 172.9,
 'iexAskSize': 504,
 'iexBidPrice': 166,
 'iexBidSize': 145,
 'iexClose': 174.927,
 'iexCloseTime': 1660807484822,
 'iexLastUpdated': 1650058113203,
 'iexMarketPercent': 0.0143634703732345,
 'iexOpen': 170.38,
 'iexOpenTime': 1681368855986,
 'iexRealtimePrice': 169.955,
 'iexRealtimeSize': 10,
 'iexVolume': 683409,
 'lastTradeTime': 1670695231131,
 'latestPrice': 167.69,
 'latestSource': 'IEX real time price',
 'latestTime': '1:11:28 PM',
 'latestUpdate': 1689813590785,
 'latestVolume': None,
 'low': 0,
 'lowSource': None

**We will split the data into cohorts of 100, followed by executing a batch API call for each of these groups.**

In [64]:
#Defining a function that splits the stocks into groups of 100.
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(trades['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings  


['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [65]:
trades[trades["Ticker"] == "VIAC"]
trades.drop(index=trades[trades['Ticker'] == 'VIAC'].index, inplace=True)

**We will improve the trades dataframe by incorporating the necessary columns.**

In [66]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
trades_data = pd.DataFrame(columns = my_columns)

#IEX_CLOUD_API_TOKEN = "token"

for symbol_string in trades["Ticker"]:
    #print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        trades_data = trades_data.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
trades_data

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,136.020,0.06561,N/A
1,AAL,17.170,-0.237057,N/A
2,AAP,209.450,0.266561,N/A
3,AAPL,169.242,0.293623,N/A
4,ABBV,156.460,0.428833,N/A
...,...,...,...,...
498,YUM,125.430,0.171764,N/A
499,ZBH,129.080,-0.224651,N/A
500,ZBRA,424.080,-0.215435,N/A
501,ZION,71.720,0.223487,N/A


## Momentum Strategy

The Momentum Strategy focuses on stocks that have exhibited the most significant price increases. To implement this strategy, we construct a dataframe containing specific columns necessary for calculating the momentum score of each stock. Later, we filter the stocks to identify those with the highest momentum scores.

In [67]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Market Capitalization',
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [68]:
len(hqm_dataframe)

0

**Batch API calls represent an efficient approach to enhance code performance and accelerate response time for requests.**

In [69]:
#Iterating  for loop to fetch the data of all S&P 500 stocks and adding them to hqm dataframe
for symbol_string in trades["Ticker"]:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['marketCap'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Market Capitalization', 'Number of Shares to Buy',
       'One-Year Price Return', 'One-Year Return Percentile',
       'Six-Month Price Return', 'Six-Month Return Percentile',
       'Three-Month Price Return', 'Three-Month Return Percentile',
       'One-Month Price Return', 'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [70]:
hqm_dataframe.head()

,Ticker,Price,Market Capitalization,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.830,40186072578,N/A,0.065679,N/A,-0.257399,N/A,-0.10967,N/A,-0.065484,N/A,N/A
1,AAL,16.890,10801320460,N/A,-0.241385,N/A,-0.177649,N/A,0.000635,N/A,-0.032292,N/A,N/A
2,AAP,212.000,12900360358,N/A,0.259343,N/A,0.005561,N/A,-0.077561,N/A,-0.145705,N/A,N/A
3,AAPL,170.799,2819395450555,N/A,0.297627,N/A,0.074142,N/A,-0.008385,N/A,-0.064668,N/A,N/A
4,ABBV,152.300,269291663315,N/A,0.435335,N/A,0.351559,N/A,0.296889,N/A,0.078275,N/A,N/A


**Sorting the trades dataframe based on One Year Price Return. We will consider the top 50 stocks.**

In [71]:

hqm_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
len(hqm_dataframe)

51

**Finding Null values and dropping them.**

In [72]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]
hqm_dataframe.dropna(axis = 0, inplace =True)
len(hqm_dataframe[hqm_dataframe.isnull().any(axis=1)].index)

0

## Calculating Momentum Percentiles

We need to calculate momentum percentile scores for the stocks, for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`


In [73]:
from scipy import stats
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile']*100)

#Print the entire DataFrame    
hqm_dataframe

0         100.0
1     98.039216
2     96.078431
3     94.117647
4     92.156863
5     90.196078
6     88.235294
7      86.27451
8     84.313725
9     82.352941
10    80.392157
11    78.431373
12    76.470588
13    74.509804
14     72.54902
15    70.588235
16    68.627451
17    66.666667
18    64.705882
19    62.745098
20    60.784314
21    58.823529
22    56.862745
23    54.901961
24    52.941176
25    50.980392
26    49.019608
27    47.058824
28    45.098039
29    43.137255
30    41.176471
31    39.215686
32    37.254902
33    35.294118
34    33.333333
35    31.372549
36    29.411765
37     27.45098
38    25.490196
39    23.529412
40    21.568627
41    19.607843
42    17.647059
43    15.686275
44     13.72549
45    11.764706
46     9.803922
47     7.843137
48     5.882353
49     3.921569
50     1.960784
Name: One-Year Return Percentile, dtype: object
0     88.235294
1         100.0
2     37.254902
3     98.039216
4      27.45098
5     96.078431
6     94.117647
7      86.27451
8     84

,Ticker,Price,Market Capitalization,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.800,22230488647,N/A,2.38082,1.0,0.855453,0.882353,0.215425,0.54902,0.081949,0.686275,N/A
1,DVN,59.300,40163359012,N/A,1.824823,0.980392,1.183068,1.0,0.484701,0.901961,0.132058,0.882353,N/A
2,NUE,144.950,39871675394,N/A,1.177532,0.960784,0.141098,0.372549,0.26925,0.647059,0.219446,0.980392,N/A
3,MRO,23.820,17589864385,N/A,1.09161,0.941176,1.048025,0.980392,0.532239,0.960784,0.113087,0.784314,N/A
4,FTNT,355.187,57121375279,N/A,0.977673,0.921569,0.097469,0.27451,0.108847,0.352941,0.122736,0.843137,N/A
5,APA,38.660,13563667421,N/A,0.935692,0.901961,1.012807,0.960784,0.502699,0.921569,0.075412,0.627451,N/A
6,FANG,140.070,24820824686,N/A,0.920196,0.882353,0.873272,0.941176,0.334308,0.72549,0.051929,0.529412,N/A
7,EOG,121.590,71582490330,N/A,0.917539,0.862745,0.839474,0.862745,0.410479,0.803922,0.028694,0.411765,N/A
8,COP,102.990,130154326893,N/A,0.898253,0.843137,0.801121,0.843137,0.428805,0.862745,0.066584,0.607843,N/A
9,CF,88.232,17910708040,N/A,0.861893,0.823529,0.858349,0.901961,0.449753,0.882353,0.113117,0.803922,N/A


### Calculating High Quantity Momentum score
It is obtained by calculating the mean of the percentile scores.



In [74]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)*100
hqm_dataframe

,Ticker,Price,Market Capitalization,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,83.800,22230488647,N/A,2.38082,1.0,0.855453,0.882353,0.215425,0.54902,0.081949,0.686275,77.941176
1,DVN,59.300,40163359012,N/A,1.824823,0.980392,1.183068,1.0,0.484701,0.901961,0.132058,0.882353,94.117647
2,NUE,144.950,39871675394,N/A,1.177532,0.960784,0.141098,0.372549,0.26925,0.647059,0.219446,0.980392,74.019608
3,MRO,23.820,17589864385,N/A,1.09161,0.941176,1.048025,0.980392,0.532239,0.960784,0.113087,0.784314,91.666667
4,FTNT,355.187,57121375279,N/A,0.977673,0.921569,0.097469,0.27451,0.108847,0.352941,0.122736,0.843137,59.803922
5,APA,38.660,13563667421,N/A,0.935692,0.901961,1.012807,0.960784,0.502699,0.921569,0.075412,0.627451,85.294118
6,FANG,140.070,24820824686,N/A,0.920196,0.882353,0.873272,0.941176,0.334308,0.72549,0.051929,0.529412,76.960784
7,EOG,121.590,71582490330,N/A,0.917539,0.862745,0.839474,0.862745,0.410479,0.803922,0.028694,0.411765,73.529412
8,COP,102.990,130154326893,N/A,0.898253,0.843137,0.801121,0.843137,0.428805,0.862745,0.066584,0.607843,78.921569
9,CF,88.232,17910708040,N/A,0.861893,0.823529,0.858349,0.901961,0.449753,0.882353,0.113117,0.803922,85.294118


### Real Value Investing Strategy

Here, we create a dataframe of specific columns inorder to calculate the real value score of the stocks and then we filter the stocks that have highest value score. For that, We use the following metrics.

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)


In [75]:
rv_columns = [
    "Ticker", 
    "Price", 
    "Num of Shares to Buy",
    "Price_to_Earnings_Ratio",
    "PE Percentile",
    "Price_to_Book_Ratio",
    "PB Percentile",
    "Price_to_Sales_Ratio",
    "PS Percentile",
    "EV/EBITDA",
    "EV/EBITDA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "RV Score"
  ]
rv_dataframe = pd.DataFrame(columns = rv_columns)


Implementing Batch API Call

In [ ]:
 
for  symbol_string in trades["Ticker"]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'    
    #print(batch_api_url)
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
        ebitda = data[symbol]["advanced-stats"]["EBITDA"]
        gross_profit = data[symbol]["advanced-stats"]["grossProfit"]
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN  
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
               [ 
                symbol, 
                data[symbol]['quote']['latestPrice'], 
                "N/A",
                data[symbol]['quote']['peRatio'],
                "N/A",
                data[symbol]["advanced-stats"]["priceToBook"],
                "N/A",
                data[symbol]["advanced-stats"]["priceToSales"],
                "N/A",
                ev_to_ebitda,
                "N/A",  
                ev_to_gross_profit,
                "N/A",
                "N/A"
                 ],
                index = rv_columns), ignore_index = True
            )
rv_dataframe        


Finding Null values and dropping them.

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
rv_dataframe.dropna(axis = 0, inplace =True)
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)


Calculating RV Score 

In [ ]:

from scipy.stats import percentileofscore as score
metrics = {
    "Price_to_Earnings_Ratio" : "PE Percentile",
    "Price_to_Book_Ratio" : "PB Percentile" ,
    "Price_to_Sales_Ratio" :  "PS Percentile",
    "EV/EBITDA" : "EV/EBITDA Percentile",
    "EV/GP" : "EV/GP Percentile"   
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric],rv_dataframe.loc[row, metric])
rv_dataframe       


In [ ]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, "RV Score"] = mean(value_percentiles)   

rv_dataframe
    

Sorting the data frame and getting TOP 50 highest Value Score stocks.

In [ ]:
rv_dataframe.sort_values("RV Score", ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]


In [ ]:
len(rv_dataframe)

In [ ]:
value_strategy.sort_values("Value Score", ascending = False, inplace = True)
value_strategy

### Backtesting the Strategy 

In [ ]:
rv_dataframe['Ticker'].isin(value_strategy['Ticker']).value_counts()

In [ ]:
hqm_dataframe['Ticker'].isin(momentum_srategy['Ticker']).value_counts()